In [1]:
import gurobipy as grb
import numpy as np
import itertools

In [2]:
import IPython.display 

In [164]:
model = grb.Model("eurobot")

Now we just take 3 heaps maximum and dispose them

   - 6 possible heaps
   - 2 funny actions
   - 1 possible tower disposal action
 

In [165]:
K, F, D, B = 6, 2, 1, 1
S =  K + F + D + B - 3
max_towers = 3

def iterer(*args):
    return itertools.product(*[range(x_) for x_ in args])

In [166]:
x = model.addVars(range(S), range(K + F + D + B), name = 'x', vtype=grb.GRB.BINARY)

In [167]:
model.remove(model.getConstrs())

## Unique action per step

In [168]:
# at each step s we can do only one action
model.addConstrs((x.sum(s, '*') == 1 for s in iterer(S)));

## Unique actions

In [169]:
model.addConstrs((x.sum('*', i) <= 1 for i in range(K+F+B+D)));

## Max 3 heaps

In [170]:
model.addConstr(grb.quicksum((x.sum('*',i) for i in iterer(K))) <= max_towers );

## Disposal after all heap picking actions

In [171]:
for s, k in iterer(S,K):
    model.addConstr( grb.quicksum( (x[s_,K+F] for s_ in range(s,S)) ) >= x[s,k] )

## Starting from Base

In [172]:
model.addConstr(x[0,K+F+D] == 1);

## Time

In [173]:
# places with cubes
heap_points = np.array([[54, 85, 0], [119, 30, 0], [150, 110, 0], [150, 190, 0], [119, 270, 0], [54, 215, 0]])

# starting place of robot
# to be redefined at the start of the algorithm
base_point  = np.array([[15, 15, 0]])

# disposal place 
# TODO: add another side of field
disposal_point = np.array([[10, 61, 0]])

# funny action places
funny_points = np.array([[10, 113, 0], [190, 10, 0]])

action_points = np.concatenate((heap_points, funny_points, disposal_point, base_point))

# time is simple distance / v
v = 20

time_travel = np.sum((action_points[np.newaxis,:]-action_points[:,np.newaxis]) ** 2, axis = 2)**0.5 / v

In [174]:
%%time
time_travel_constr_gen = (x[s, k_1] * x[s+1, k_2] * time_travel[k_1, k_2] for  s,k_1, k_2 in 
                          #[(0,9, 2)])
                          iterer(S-1,K+F+D+B,K+F+D+B))
left_part_time_1 = grb.quicksum(time_travel_constr_gen)

left_part_time = left_part_time_1

CPU times: user 24 ms, sys: 4 ms, total: 28 ms
Wall time: 25.1 ms


In [141]:
ξ = model.addVars(range(K), name = 'xi', vtype=grb.GRB.BINARY)
model.addConstrs((x.sum('*',k) >= ξ[k] for k in range(K))) 
model.addConstr(ξ.sum('*') <= x.sum('*',K+F)*max_towers) # get points only if disposed

<gurobi.Constr *Awaiting Model Update*>

In [175]:
Time = 200.0
Points = 220.0

In [176]:
total_points = grb.quicksum(x.sum('*',k) for k in range(K))*45 + x.sum('*',K)*30 + x.sum('*',K+1)*55

## Objective - points maximize

In [50]:
model.addConstr(left_part_time <= Time, 'time')
model.setObjective(total_points, grb.GRB.MAXIMIZE)

## Objective - time minimize

In [177]:
model.addConstr(x.sum('*',2) == 0)

<gurobi.Constr *Awaiting Model Update*>

In [178]:
model.addConstr(total_points == Points, 'points')
model.setObjective(left_part_time, grb.GRB.MINIMIZE)

In [179]:
model.Params.TimeLimit = 300.0
model.update()
model.optimize()

Changed value of parameter TimeLimit to 300.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 63 rows, 70 columns and 456 nonzeros
Model has 540 quadratic objective terms
Variable types: 0 continuous, 70 integer (70 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 26 rows and 27 columns
Presolve time: 0.00s
Presolved: 282 rows, 288 columns, 999 nonzeros
Variable types: 0 continuous, 288 integer (288 binary)
Found heuristic solution: objective 42.0238872

Root relaxation: objective 2.612314e+00, 43 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    2.61231    0   26   42.02389    2.61231  93.8%     -    0s
     0     0    4.11903    0   22   42.02389 

In [180]:
model.printAttr('x')


    Variable            x 
-------------------------
      x[0,9]            1 
      x[1,1]            1 
      x[2,7]            1 
      x[3,3]            1 
      x[4,0]            1 
      x[5,8]            1 
      x[6,6]            1 
